In [ ]:
!pip install Flask pyngrok line-bot-sdk requests openai --quiet

In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
openai_api_key = userdata.get('OPENAI_API_KEY')
port = 5000
print(line_channel_access_token)
print("====")
print(line_channel_secret)

zKPBoejRjkFsB4q3iXWFQwMCybXeUorzi4Vhu5fa3ag+DwGtyCEK4bNplKRajqtED3u6iztG8boNYNX5ZbGyghZvkTp1ntIW72aPG9GWgaLuvFuxn6APOy7lEYTGo/gShXR8ZEzax961ZizVchxtcgdB04t89/1O/w1cDnyilFU=
====
01d18cc5b05062dd6fd8e3fecafb315d


In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5000, name="linebot_tunnel")
print(tunnel.public_url)

https://79a3-35-194-86-198.ngrok-free.app


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)
def get_completion(prompt):
    messages = [{"role": "system", "content": "以下用繁體中文台灣用語回答"},
                {"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
        max_tokens=256

    )
    return response.choices[0].message.content

In [ ]:
result = get_completion("簡介東吳大學")
print(result)

東吳大學是一所位於台灣台北市的知名私立大學，創立於1928年，是台灣歷史悠久的學府之一。東吳大學以人文學科為主，並擁有豐富的師資和研究資源。校園環境優美，教學設備完善，並且積極推動國際交流與合作。東吳大學在台灣享有很高的聲譽，許多知名的學者、政治人物和商業領袖都是該校的校友。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    StickerMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = get_completion(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )


@handler.add(MessageEvent, message=StickerMessageContent)
def handle_sticker_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[StickerMessage(
                    package_id=event.message.package_id,
                    sticker_id=event.message.sticker_id)
                ]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(
                    title='定位訊息',
                    address=event.message.address,
                    latitude=event.message.latitude,
                    longitude=event.message.longitude
                )]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/May/2024 18:36:16] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}
BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"510033541321719937","quoteToken":"OpDNv0WvnDQpgrtfpIErmLFqOKzP_6vAcIRNBKqthw6QeSIWp64GEOIm7mVcQS6WhywZ7PI6PShtJ3uVSJd1Dk56Uhvbei49c0gtiff9eO-JZQ-FjQRi0wEdSihixMfOcuS7Tp1tkyx3NYmgGpzK6w","text":"AI 東吳大學"},"webhookEventId":"01HYXNDRRCMQSNH8REMCXQK8X0","deliveryContext":{"isRedelivery":false},"timestamp":1716834984557,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"6fefdaeac1644b639584b8f7ef429cf6","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [27/May/2024 18:36:28] "POST / HTTP/1.1" 200 -
